## Lab01: Collect and preprocess data

- Name: Dương Trường Bình
- Student code: 21127229

***
## How to do and submit your assignment

**Work on your assignment**

You will do your assignment directly in this notebook. First, fill in your name and ID at the beginning of the file. In the notebook, fill in places that say:
```python
#TODO
```

During your work, you can print out the result, create more cells to test, or create more functions to handle things. Please note that <font color=red>you are not allowed to delete or modify my code cells</font> (except in the case that mentioned above). Let remove `raise NotImplementedError("not implement")` when running code.

Always press `Ctrl + S` in order to save your work.

**Notes:**

    *  Copy means zero
    *  You have to submit your work on time. No exception
    *  Any questions about your grade are based on the assignment submitted on Moodle
    *  Wrong submission takes you -2 points

**Submission guideline**

When grading your assignment, I will choose `Kernel` - `Restart Kernel & Run All Cells` in order to restart the kernel and run all cells in your notebook. Therefore, you should do that before submitting to ensure that the outputs are all as expected.

After that, you make a submited direction as follow:

- Folder `StudentCode` (e.g. If your student code is 1234567, then your folder is `1234567`)
    - File `<StudentCode>.ipynb` (e.g. If your student code is 1234567, then your file is `1234567.ipynb`)

Finally, you compress your folder (`StudentCode`) and submit on Moodle. **The extension of the file is nothing else but `.zip`.**

<font color=red>Please strictly follow the submission rules.</font>

# 1. Set-up environment

In [1]:
#Necessary Packages
import time
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
# YOUR CODE HERE (OPTION)
import re

# If you need other support packages

# 2. Collect data from a website by parsing HTML (3p)

In this section, you are going to collect data from a website simulating the sale of Pokemon. I have prepared all the needed links in a file (`pokemon.txt`), and you have to crawl data from these links. The expected output is a `dataframe` with the following fields:

+ `SKU`: ID of Pokemon
+ `Name`: Name of Pokemon
+ `Price`: Price of Pokemon
+ `InStock`: Quantity of Pokemons in stock
+ `Categories`: The category of Pokemon
+ `Tags`: Tags of Pokemon

Your mission is to complete function `collect_data` with `course_urls_file` as your input parameter. The output should look like `pokemon_example.csv` (I list out some examples so you can easily imagine your work).

In [2]:
pokemon_example = pd.read_csv('pokemon_example.csv')
pokemon_example

,SKU,Name,Price,InStock,Categories,Tags
0,4391,Bulbasaur,63.0,45,"Pokemon, Seed","bulbasaur, Overgrow, Seed"
1,7227,Ivysaur,87.0,142,"Pokemon, Seed","ivysaur, Overgrow, Seed"
2,7036,Venusaur,105.0,30,"Pokemon, Seed","Overgrow, Seed, venusaur"
3,9086,Charmander,48.0,206,"Lizard, Pokemon","Blaze, charmander, Lizard"
4,6565,Charmeleon,165.0,284,"Flame, Pokemon","Blaze, charmeleon, Flame"


In [3]:
def collect_data(course_urls_file):
    # load paths from file
    url_file = open(course_urls_file)
    urls = url_file.readlines()
    urls_filtered = [item[:-1] for item in urls]

    # init empty list to store the values of each attribute.
    SKU = []
    Names = []
    Prices = []
    InStocks = []
    Categories = []
    Tags = []

    session = requests.Session()
    for idx, url in enumerate(urls_filtered):
        # TODO
        print(
            f'Collecting data from pokemon {idx + 1}: {(idx + 1) / len(urls_filtered) * 100:.2f}%', end='\r', flush=True)

        response = session.get(url)
        # Parse the HTML content of the page using Beautiful Soup
        soup = BeautifulSoup(response.text, 'html.parser')

        summary_div_tag = soup.find('div', class_='summary entry-summary')

        name = summary_div_tag.find('h1')
        price_tag = summary_div_tag.find('p', class_='price')
        stock_tag = summary_div_tag.find('p', class_='stock in-stock')
        product_meta_tag = summary_div_tag.find('div', class_='product_meta')
        sku_tag = product_meta_tag.find('span', class_='sku_wrapper')
        categories_tag = product_meta_tag.find('span', class_='posted_in')
        tags_tag = product_meta_tag.find('span', class_='tagged_as')

        name = name.text.strip() if name else ''
        price = float(re.search(r'\d+\.\d+', price_tag.text.strip()
                                ).group())
        stock = int(re.search(r'\d+', stock_tag.text.strip()).group())
        sku = int(sku_tag.find('span').text.strip())
        categories = [a.text.strip() for a in categories_tag.find_all('a')]
        tags = [a.text.strip() for a in tags_tag.find_all('a')]

        Names.append(name)
        Prices.append(price)
        InStocks.append(stock)
        SKU.append(sku)
        Categories.append(categories)
        Tags.append(tags)
        

    Categories = [', '.join(lst) for lst in Categories]
    Tags = [', '.join(lst) for lst in Tags]

    data = pd.DataFrame({"SKU": SKU,
                         "Name": Names,
                         "Price": Prices,
                         "InStock": InStocks,
                         "Categories": Categories,
                         "Tags": Tags})

    return data

In [4]:
#TEST
data_pokemon = collect_data("pokemon.txt")
assert data_pokemon.shape == (755, 6)

In [5]:
# Save to csv file with name pokemon.csv
# TODO
data_pokemon.to_csv('pokemon.csv', index=False)

In [6]:
# Test read file
pokemon_test= pd.read_csv('pokemon.csv')
pokemon_test.head()

,SKU,Name,Price,InStock,Categories,Tags
0,4391,Bulbasaur,63.0,45,"Pokemon, Seed","bulbasaur, Overgrow, Seed"
1,7227,Ivysaur,87.0,142,"Pokemon, Seed","ivysaur, Overgrow, Seed"
2,7036,Venusaur,105.0,30,"Pokemon, Seed","Overgrow, Seed, venusaur"
3,9086,Charmander,48.0,206,"Lizard, Pokemon","Blaze, charmander, Lizard"
4,6565,Charmeleon,165.0,284,"Flame, Pokemon","Blaze, charmeleon, Flame"


# 3. Collect data using Web API (4p)

In this section, your work is to practice to crawl data using Web API (http://api.worldbank.org/v2/country/all/indicator/SP.POP.TOTL). This is the data of World Bank which includes demographic data and other statistics related to Population, Employment, Health, GDP, Energy Consumption,... for all countries in the world from 1960 to 2022.

From the following selected indicators:
- `SP.POP.TOTL` - Total population
- `SP.POP.TOTL.FE.IN` - Total female population
- `SP.POP.TOTL.MA.IN` - Total male population
- `SP.DYN.CBRT.IN` - Birth rate
- `SP.DYN.CDRT.IN` - Death rate
- `SP.DYN.LE00.MA.IN` - Average life expectancy of male
- `SP.DYN.LE00.FE.IN` - Average life expectancy of female
- `SE.PRM.ENRR` - Primary school enrollment rate
- `SE.TER.ENRR` - High school enrollment rate
- `SE.PRM.CMPT.ZS` - Primary completion rate
- `SE.ADT.1524.LT.ZS` - Literacy rate of people ages 15-24

You are required to collect data from 7 countries and save to dataframe `data_countries`:
- `US` - United States of America
- `IN` - India
- `CN` - China
- `JP` - Japan
- `CA` - Canada
- `GB` - Great Britain
- `ZA` - South Africa

You can expand your work on collecting data (such as collecting data from other countries and other indicators) by reading: https://datahelpdesk.worldbank.org/knowledgebase/articles/889392-api-documentation

**Hints**:

- Use the based URL: http://api.worldbank.org/v2/
- In order to collect data for each indicator of each country, you can use the URL: "http://api.worldbank.org/v2/countries/{country_code}/indicators/{indicator_code}"
    + `country_code` and `indicator_code` are provided above.
    + For example, you can use the following URL to get the `Total population` of Japan: http://api.worldbank.org/v2/countries/jp/indicators/SP.POP.TOTL

In [7]:
data_countries_examples = pd.read_csv("countries_example.csv")
data_countries_examples

,Total Population,Female Population,Male Population,Birth Rate,Death Rate,Male life expectancy,Female life expectancy,"School enrollment, primary","School enrollment, tertiary",Primary completion rate,Literacy rate,Year,Country
0,333287557.0,168266219.0,165021339.0,None,None,None,None,None,None,None,None,2022,USA
1,332031554.0,167550001.0,164481553.0,11.0,10.4,73.5,79.3,None,None,None,None,2021,USA
2,331511512.0,167203010.0,164308503.0,10.9,10.3,74.2,79.9,100.305793762207,87.5676574707031,100.923667907715,None,2020,USA
3,328329953.0,165599805.0,162730147.0,11.4,8.7,76.3,81.4,100.981300354004,87.8887100219727,100.489051818848,None,2019,USA
4,326838199.0,164926348.0,161911851.0,11.6,8.678,76.2,81.2,101.256561279297,88.2991790771484,100.092697143555,None,2018,USA
5,325122128.0,164151818.0,160970309.0,11.8,8.638,76.1,81.1,101.821441650391,88.1673889160156,98.8321990966797,None,2017,USA
6,323071755.0,163224028.0,159847727.0,12.2,8.493,76.1,81.1,101.362861633301,88.8350524902344,None,None,2016,USA
7,320738994.0,162158414.0,158580581.0,12.4,8.44,76.3,81.2,100.299911499023,88.8894119262695,None,None,2015,USA
8,318386329.0,161084758.0,157301571.0,12.5,8.237,76.5,81.3,99.6733779907227,88.6268692016602,None,None,2014,USA
9,316059947.0,160034189.0,156025758.0,12.4,8.215,76.4,81.2,99.455436706543,88.7264175415039,None,None,2013,USA


In [8]:
BASE_URL = 'http://api.worldbank.org/v2/'
COUNTRIES = ["US", "IN", "CN", "JP", "CA", "GB", "ZA"]
INDICATORS = ['SP.POP.TOTL',
             'SP.POP.TOTL.FE.IN',
             'SP.POP.TOTL.MA.IN',
             'SP.DYN.CBRT.IN',
             'SP.DYN.CDRT.IN',
             'SP.DYN.LE00.MA.IN',
             'SP.DYN.LE00.FE.IN',
             'SE.PRM.ENRR',
             'SE.TER.ENRR',
             'SE.PRM.CMPT.ZS',
             'SE.ADT.1524.LT.ZS']

#TODO (option)
# If you need other initializations

In [9]:
def collect_data(countryCode, per_page, start_year, end_year):
    # TODO

    country_data = {
        'Total Population': [],
        'Female Population': [],
        'Male Population': [],
        'Birth Rate': [],
        'Death Rate': [],
        'Male life expectancy': [],
        'Female life expectancy': [],
        'School enrollment, primary': [],
        'School enrollment, tertiary': [],
        'Primary completion rate': [],
        'Literacy rate': [],
        'Year': [],
        'Country': []
    }
    for indicator in INDICATORS:
        url = BASE_URL + \
            f'countries/{countryCode}/indicators/{indicator}?format=json&per_page={per_page}&date={start_year}:{end_year}'
        response = requests.get(url)
        # Parse the JSON response
        json_data = response.json()

        list_data = json_data[1]
        for entry in list_data:
            value = entry['value']

            if indicator == 'SP.POP.TOTL':
                country_data['Country'].append(entry["countryiso3code"])
                country_data['Year'].append(int(entry['date']))
                country_data['Total Population'].append(float(value))
            elif indicator == 'SP.POP.TOTL.FE.IN':
                country_data['Female Population'].append(float(value))
            elif indicator == 'SP.POP.TOTL.MA.IN':
                country_data['Male Population'].append(float(value))
            elif indicator == 'SP.DYN.CBRT.IN':
                country_data['Birth Rate'].append(value)
            elif indicator == 'SP.DYN.CDRT.IN':
                country_data['Death Rate'].append(value)
            elif indicator == 'SP.DYN.LE00.MA.IN':
                country_data['Male life expectancy'].append(value)
            elif indicator == 'SP.DYN.LE00.FE.IN':
                country_data['Female life expectancy'].append(value)
            elif indicator == 'SE.PRM.ENRR':
                country_data['School enrollment, primary'].append(value)
            elif indicator == 'SE.TER.ENRR':
                country_data['School enrollment, tertiary'].append(value)
            elif indicator == 'SE.PRM.CMPT.ZS':
                country_data['Primary completion rate'].append(value)
            elif indicator == 'SE.ADT.1524.LT.ZS':
                country_data['Literacy rate'].append(value)
    dataframe = pd.DataFrame(country_data)
    dataframe = dataframe.astype(object).replace(np.nan, 'None')
    return dataframe

In [10]:
def Generate_Countries_Dataset(countryCode_list):
    data = pd.DataFrame()
    for countryCode in countryCode_list:
        data = pd.concat([data, collect_data(countryCode = countryCode, per_page = 100, start_year = 2000, end_year = 2022)], axis=0)
        
    return data

In [11]:
#TEST
data_countries = Generate_Countries_Dataset(COUNTRIES)
assert data_countries.shape == (161, 13)

In [12]:
# Save to csv file with name coutries.csv
#TODO
data_countries.to_csv('countries.csv',index=False)

In [13]:
# Test read file
countries_test = pd.read_csv('countries.csv')
countries_test.head(10)

,Total Population,Female Population,Male Population,Birth Rate,Death Rate,Male life expectancy,Female life expectancy,"School enrollment, primary","School enrollment, tertiary",Primary completion rate,Literacy rate,Year,Country
0,333287557.0,168266219.0,165021339.0,None,None,None,None,None,None,None,None,2022,USA
1,332031554.0,167550001.0,164481553.0,11.0,10.4,73.5,79.3,None,None,None,None,2021,USA
2,331511512.0,167203010.0,164308503.0,10.9,10.3,74.2,79.9,100.305793762207,87.5676574707031,100.923667907715,None,2020,USA
3,328329953.0,165599805.0,162730147.0,11.4,8.7,76.3,81.4,100.981300354004,87.8887100219727,100.489051818848,None,2019,USA
4,326838199.0,164926348.0,161911851.0,11.6,8.678,76.2,81.2,101.256561279297,88.2991790771484,100.092697143555,None,2018,USA
5,325122128.0,164151818.0,160970309.0,11.8,8.638,76.1,81.1,101.821441650391,88.1673889160156,98.8321990966797,None,2017,USA
6,323071755.0,163224028.0,159847727.0,12.2,8.493,76.1,81.1,101.362861633301,88.8350524902344,None,None,2016,USA
7,320738994.0,162158414.0,158580581.0,12.4,8.44,76.3,81.2,100.299911499023,88.8894119262695,None,None,2015,USA
8,318386329.0,161084758.0,157301571.0,12.5,8.237,76.5,81.3,99.6733779907227,88.6268692016602,None,None,2014,USA
9,316059947.0,160034189.0,156025758.0,12.4,8.215,76.4,81.2,99.455436706543,88.7264175415039,None,None,2013,USA
